In [1]:
%%configure -f
{
 "executorCores": 4, 
 "executorMemory": "47696M",
 "conf": {"spark.default.parallelism": 1000,
          "spark.sql.shuffle.partitions": 1000,
          "spark.task.cpus": 1
         }
}

## TODO:
* extend company_description matrix to include all company info
* check if this does a better job with recall just with SVD then revenue-ebitda reranking

In [2]:
model_path = "s3://onai-ml-dev-eu-west-1/company2vec/model_description_only/svd"
word2id_path = "s3://onai-ml-dev-eu-west-1/company2vec/common"

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
2,application_1568197055316_0003,pyspark,idle,,,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
from gensim.utils import lemmatize, tokenize
from gensim.parsing.preprocessing import remove_stopwords
from gensim.parsing.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

from langdetect import detect
import string
from itertools import islice
import numpy as np
import smart_open
from collections import Counter, defaultdict
import math
import json

from pyspark.ml.feature import (HashingTF,
                                IDF,
                                Tokenizer,
                                StopWordsRemover,
                                CountVectorizer,
                                StringIndexer,
                                OneHotEncoderEstimator,
                                VectorAssembler,
                                VectorSizeHint,
                                StandardScaler,
                                PCA
                               )
from pyspark.ml import Pipeline, Transformer

from pyspark.sql import SparkSession
import pyspark.sql.functions as F
import pyspark.sql.types as T

from pyspark.mllib.linalg import DenseMatrix
from pyspark.mllib.linalg.distributed import RowMatrix
from pyspark.mllib.linalg import Vector as MLLibVector, Vectors as MLLibVectors

from pyspark.ml.linalg import Vectors, VectorUDT

import smart_open
import faiss
import requests

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
import graphene

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
import os
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search, Index
import numpy as np
import ssl
from elasticsearch.connection import create_ssl_context
from elasticsearch_dsl import Q

import urllib3
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

public_labels = {
    2444485: [9429145, 93196, 9870522, 7914436],
    645782: [380011, 392154, 5523392],
    875173: [237655379, 931146, 418171],
    813607: [100231, 357076],
    380011: [357076, 324490, 93339],
    882889: [127202, 5478606, 1025390, 645782],
    46329052: [189096, 915379, 46895276, 877008, 325290136, 20524024, 271958947, 21852987, 26363560, 110104150],
    106335: [319676, 377732, 61206100],
    877769: [874042, 780678, 953488, 883809, 875295, 874186, 874119],
    34049884: [882155, 30428758, 315394, 23037669, 27561],
    254287477: [883300, 30614595],
    5600108: [285880, 5433540, 878697, 35650, 688262, 226852452],
    876031: [410182, 874470, 874191, 879732, 5395336],
    883752: [880697, 65340486, 26320074, 883327, 1034090, 257501324],
    5920885: [1494039, 268074105, 34534627, 20385800, 23000545, 124640],
    628413: [272054403, 91192, 309779, 140283, 138644],
    364040: [381388, 184945, 874170, 42751952, 874183, 314896, 5126590, 841504],
    257501324: [35000, 47320264, 253748612, 85076655, 32053, 12144785, 8186273, 9934160, 557267859],
    695204: [35303, 274561, 683719, 370857, 561001, 874022, 387473, 394038, 8274485, 12188205],
    30614595: [883300, 254287477, 9956099, 380011, 27868703, 2386697, 126857],
    28224119: [26824144, 35023689, 386639, 393661],
    32449506: [875260, 27169270, 5629762, 26014489, 286119],
    233324810: [874864, 159230, 27860587, 35806, 876981],
    879554: [5487000, 236715563, 412090459, 875192, 278679, 180871, 22516334, 30274893, 5478907],
    5580060: [118474533, 1779941, 265154, 10405454],
    23335317: [7885406, 277444, 278933, 8983678, 874143, 409119],
    381865672: [874842, 410366, 873649, 275789, 882473],
    937352: [876758, 879422, 128861678, 6461781, 1859063],
    874119: [881803, 875849, 231533, 877769, 780678, 953488, 875295, 874042, 775001, 874186],
    680934: [135398, 882299, 668578, 4481676, 32012],
    2248076: [141249, 4975204, 98876, 21828553],
    3606442: [882547, 4509042, 20703565, 7435035, 94799, 288033, 359868],
    877235: [295170, 175265, 874520, 410366, 873649, 874977, 167945, 8090046],
    84148802: [275789, 30339992, 5533238, 5718736],
    5523392: [645782, 11809880, 1353107, 962864],
    413744: [409932, 875491, 109303666, 91638],
    314896: [330589, 34768, 184945],
    5126590: [874855, 631781, 364040, 831357, 874170],
    377732: [319676, 106772, 106335, 704634, 320105, 874828, 873861, 1519242, 533853947],
    874652: [377732, 319676, 106772, 704634, 312375, 278933, 874828, 4863668]
}
HOST = "https://vpc-datalake-es-dev-pkpwahah2cv4owesrl4g5xzafe.eu-west-2.es.amazonaws.com:443"
INDEX = "company"

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
word2id = {}
id2word = {}
with smart_open.open(f"{word2id_path}/bow/word2id.csv", "r") as f:
    for line in f:
        word,idd = line.strip().split(",")
        word2id[word] = int(idd)
        id2word[int(idd)] = word

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
with smart_open.open(f"{model_path}/V/mat.np", "rb") as f:
    V = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
with smart_open.open(f"{model_path}/X/X_pca.np", "rb") as f:
    X_pca = np.load(f)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
with smart_open.open(f"{model_path}/X/company_info.np", "rb") as f:
    description = np.load(f, allow_pickle=True)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
company2id = {}
for i,desc in enumerate(description):
    company2id[str(desc[0])] = i

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
description[0]

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

array([19591, 'Chanin Capital Partners',
       'Chanin Capital Partners is a boutique investment-banking firm that provides financial advisory services. The firm offers chapter 11 restructurings; out-of-court reorganizing; recapitalization; exchange offers and consent consolidation; exit financing; strategic alternative; and working capital and cash management consulting services. Additionally, it provides merger and acquisition and transaction advisory services. Chanin Capital Partners\x92 clientele include Dura Automotive Systems, Calpine, Home Interiors & Gifts, Inc., and Adelphia. The firm was founded in 1984 and is based in Los Angeles, California with additional offices in New York City and London, United Kingdom. As of October 31, 2006, Chanin Capital Partners operates as a subsidiary of Duff & Phelps Corporation.',
       'United States', '100960', None], dtype=object)

In [7]:
def find_closest_revenue(candidate, revenue, ebitda):
    if candidate.revenue == 0 or candidate.ebitda == 0:
        return candidate.score * 100

    if candidate.revenue < revenue:
        score_revenue = 1 - candidate.revenue/revenue
    else:
        score_revenue = 1 - revenue/candidate.revenue

    if candidate.ebitda < ebitda:
        score_ebitda = 1 - candidate.ebitda/ebitda
    else:
        score_ebitda = 1 - ebitda/candidate.ebitda

    return (score_ebitda + score_revenue) / 2


def revenue_reranker(results):
    query = results[0]
    return sorted(results[1:21],
                  key=lambda x: find_closest_revenue(x,
                                                     query.revenue,
                                                     query.ebitda
                                                     )
                  )

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
class Company(graphene.ObjectType):
    id = graphene.ID()
    name = graphene.String()
    country = graphene.String()
    sector = graphene.String()
    revenue = graphene.Float()
    ebitda = graphene.Float()
    num_employees = graphene.Int()
    score = graphene.String()
    region = graphene.String()
    description = graphene.String()
    company_type = graphene.String()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
client = Elasticsearch(HOST+"/")

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
n,d = X_pca.shape

In [ ]:
index = faiss.IndexFlatL2(d)
index.add(X_pca)
print(index.ntotal)

In [14]:
def obtain_result(doc_id, 
                  reranker=lambda ret: ret, 
                  max_size=100, 
                  region=None, 
                  keywords=None,
                  company_type=None,
                  revenue_min=-1,
                  revenue_max=-1
                 ):

    # for key, val in query['_source'].items():
    #     print("%s: %s" % (key, str(val)))

    

        results.append(
            Company(
                id=id,
                name=cname,
                revenue=revenue,
                ebitda=ebitda,
                sector=sector,
                num_employees=num_employees,
                country=country,
                score=score,
                region=region,
                description=description,
                company_type=company_type,
            )
        )

    return reranker(results)


def find_peer_in_result(results, peer):
    for i, company in enumerate(results):
        if peer == int(company.id):
            return i+1

    return -1


def add_ranks_in_tops(ranks, counts):
    for rank in ranks:
        if 0 < rank <= 10:
            counts[0] += 1
        elif 10 < rank <= 20:
            counts[1] += 1
        elif 20 < rank <= 50:
            counts[2] += 1
        elif 50 < rank <= 100:
            counts[3] += 1
        elif 100 < rank:
            counts[4] += 1
        else:
            counts[5] += 1
    return counts

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### More like this, revenue re-ranking

In [15]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, revenue_reranker)
    ranks = []
    
    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)

    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 1, 7, -1, 9]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 9, 1]

[380011, 2, 5, -1]

[882889, -1, -1, -1, 12]

[46329052, 8, 15, -1, -1, 2, -1, 9, -1, -1, -1]

[106335, -1, 9, 2]

[877769, 4, 2, 1, 15, -1, -1, -1]

[34049884, 3, 16, -1, -1, 10]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, 6, 9, 3, 15, 17]

[883752, 17, -1, -1, -1, -1, -1]

[5920885, -1, -1, 17, 12, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, -1, -1, -1, 4, -1, 12, -1, -1]

[257501324, 11, -1, -1, -1, -1, -1, -1, -1, -1]

[695204, 1, 10, 13, 19, 16, 6, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, 4, -1, -1]

[233324810, -1, -1, 20, 11, 5]

[879554, -1, -1, 19, 13, -1, -1, -1, -1, -1]

[5580060, -1, 18, -1, -1]

[23335317, 4, 14, -1, 7, 20, -1]

[381865672, -1, -1, 11, 5, 8]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1]

[680934, -1, -1, -1, -1, -1]

[2248076, -1, 12, -1, -1]

[3606442, 

### More like this, revenue re-ranking, 500

In [16]:
tops = [0] * 6 # top 10, 20, 50, 100, >100, -1

mrr = 0
count = 0

ranks_default = []
missed_peers_500 = defaultdict(list)
for public_label,peers in public_labels.items():
    query_id = public_label
    results = obtain_result(query_id, revenue_reranker, max_size=500)
    ranks = []

    for peer in peers:
        peer_location = find_peer_in_result(results, peer)
        ranks.append(peer_location)
        ranks_default.append(peer_location)
        if peer_location == -1:
            missed_peers_500[public_label].append(peer)

    tops = add_ranks_in_tops(ranks, tops)

    mrr += sum([1/rank for rank in ranks if rank > 0])
    count += sum(1 for rank in ranks if rank > 0)

    ranks.insert(0, query_id)

    print(ranks)
    print()

print(tops)
print(np.mean([1/rank for rank in ranks_default if 0 < rank <= 20]))
print()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[2444485, 1, 7, -1, 9]

[645782, -1, -1, -1]

[875173, -1, -1, -1]

[813607, 9, 1]

[380011, 2, 5, -1]

[882889, -1, -1, -1, 12]

[46329052, 8, 15, -1, -1, 2, -1, 9, -1, -1, -1]

[106335, -1, 9, 2]

[877769, 4, 2, 1, 15, -1, -1, -1]

[34049884, 3, 16, -1, -1, 10]

[254287477, -1, -1]

[5600108, -1, -1, -1, -1, -1, -1]

[876031, 6, 9, 3, 15, 17]

[883752, 17, -1, -1, -1, -1, -1]

[5920885, -1, -1, 17, 12, -1, -1]

[628413, -1, -1, -1, -1, -1]

[364040, -1, -1, -1, 4, -1, 12, -1, -1]

[257501324, 11, -1, -1, -1, -1, -1, -1, -1, -1]

[695204, 1, 10, 13, 19, 16, 6, -1, -1, -1, -1]

[30614595, -1, -1, -1, -1, -1, -1, -1]

[28224119, -1, -1, -1, -1]

[32449506, -1, -1, 4, -1, -1]

[233324810, -1, -1, 20, 11, 5]

[879554, -1, -1, 19, 13, -1, -1, -1, -1, -1]

[5580060, -1, 18, -1, -1]

[23335317, 4, 14, -1, 7, 20, -1]

[381865672, -1, -1, 11, 5, 8]

[937352, -1, -1, -1, -1, -1]

[874119, -1, 1, -1, -1, -1, -1, -1, -1, -1, -1]

[680934, -1, -1, -1, -1, -1]

[2248076, -1, 12, -1, -1]

[3606442, 

In [127]:
def debug_es(target):
    
    filters = Q('match', company_id=target)
    for peer in public_labels[target]:
        filters |= Q('match', company_id=peer)
    
    q = Q('more_like_this',
          like=[{"_id": target, "_index": INDEX}],
          fields=[
              "industry_class_code_id.keyword",
              "company_description"
              ],
          min_doc_freq=1,
          min_term_freq=0,
          max_query_terms=15,
          include=True, )

    s_no_filter = (
        Search(index=INDEX)
        .using(client)
    )
    
    s = (
        Search(index=INDEX)
        .using(client)
        .filter(filters)
    )

    q = Q('bool',
          should=[q]
          )

    s = (s
         .query(q)
         .params(search_type='dfs_query_then_fetch')
         .extra(explain=True)
         ).execute()
    
    s_no_filter = (
        s_no_filter
        .query(q)
        .params(search_type='dfs_query_then_fetch')
        .extra(explain=True)
        ).execute()

    
    def get_keyword(kw_s):
        return kw_s.split(":")[1].split()[0]
    
    hit = s[0:1][0]
    ret = []
    print("#"*40, hit.company_name, "#"*40)
    for el in hit.meta.explanation.details[0].details:
        ret.append((el.value,
                    get_keyword(el.description),
                    el.details[0].details[1].value,
                    el.details[0].details[0].value
                   ))
    print(f"Score: {hit.meta.explanation.details[0].value}")
    for match, term, tf, idf in sorted(ret, reverse=True):
        print(f"{match} {term}, TF:{tf}, IDF:{idf}")
    print("-"*40)
    
    for rank,(hit, hit_no_filter) in enumerate(zip(s[1:100], s_no_filter[1:100])):
        print("-"*40, rank+1, "-"*40)
        print("Actual peer")
        ret = []
        print(f"Score: {hit.meta.explanation.details[0].value}")
        for el in hit.meta.explanation.details[0].details:
            ret.append((el.value, 
                        get_keyword(el.description),
                        el.details[0].details[1].value,
                        el.details[0].details[0].value
                       ))
        for match, term, tf, idf in sorted(ret, reverse=True):
            print(f"{match} {term}, TF:{tf}, IDF:{idf}")
        print("-"*40)
        
        print("ES peer")
        ret = []
        print(f"Score: {hit_no_filter.meta.explanation.value}")
        for el in hit_no_filter.meta.explanation.details:
            ret.append((el.value, 
                        get_keyword(el.description),
                        el.details[0].details[1].value,
                        el.details[0].details[0].value
                       ))
        for match, term, tf, idf in sorted(ret, reverse=True):
            print(f"{match} {term}, TF:{tf}, IDF:{idf}")
        print("-"*40)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [128]:
for target in islice(public_labels, 1):
    debug_es(target)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

######################################## AT & S Austria Technologie & Systemtechnik Aktiengesellschaft ########################################
Score: 96.614075
12.863629 pcbs, TF:1.3823372, IDF:9.30571
9.222403 hdi, TF:0.9242983, IDF:9.977735
8.178932 100488, TF:1.0, IDF:8.178932
7.7163377 systemtechnik, TF:0.7165479, IDF:10.768767
7.3195224 substrates, TF:0.9242983, IDF:7.9190044
6.7066627 technologie, TF:0.7165479, IDF:9.359685
5.9041557 flexible, TF:0.9242983, IDF:6.387717
5.66094 circuit, TF:0.9242983, IDF:6.1245813
5.446132 rigid, TF:0.7165479, IDF:7.6005135
4.9172525 austria, TF:0.9242983, IDF:5.3199844
4.6926346 aktiengesellschaft, TF:0.7165479, IDF:6.5489473
4.4797482 printed, TF:0.7165479, IDF:6.2518477
4.288034 boards, TF:0.7165479, IDF:5.984295
4.252323 standard, TF:0.7165479, IDF:5.934458
4.1053734 copper, TF:0.7165479, IDF:5.7293773
0.85999984 and, TF:1.506786, IDF:0.57075113
----------------------------------------
---------------------------------------- 1 -------------

In [34]:
for target,peers in missed_peers.items():
    print("#"*40, f"Target {target}", "#"*40)
    
    hit = client.get(index='company', id=target)
    cname = hit["_source"]["company_name"]
    sector = hit["_source"].get("industry_class_code_desc", "")

    country =  hit["_source"].get("country", "")
    company_type = hit["_source"].get("company_type", "")

    revenue = hit["_source"].get("latest_revenue", 0)
    num_employees = hit["_source"].get("number_of_employees", 0)
    region = hit["_source"].get("region", "n/a")

    description = hit["_source"].get("company_description", "")
    ebitda = hit["_source"].get("latest_ebitda", 0)
    
    for el in debug_es(target)["explanations"][0]["explanation"].split():
        print(el)

    print(cname)
    print(description)
    print(revenue)
    print(num_employees)
    print(company_type)
    print(sector)
    print(region)
    print("-"*40)
    
    results = []
    for peer in peers:
        

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

######################################## Target 2444485 ########################################
(company_description:copper
company_description:standard
company_description:boards
company_description:printed
company_description:aktiengesellschaft
company_description:rigid
company_description:and
company_description:austria
company_description:technologie
company_description:circuit
company_description:flexible
company_description:systemtechnik
company_description:substrates
company_description:hdi
company_description:pcbs
industry_class_code_desc.keyword:Printed
circuit
boards)~4
AT & S Austria Technologie & Systemtechnik Aktiengesellschaft
AT & S Austria Technologie & Systemtechnik Aktiengesellschaft, together with its subsidiaries, manufactures and distributes printed circuit boards (PCBs) worldwide. It operates through three segments: Mobile Devices & Substrates; Automotive, Industrial, Medical; and Others. The company offers double sided standard, metal core, and copper inlay PCBs

In [35]:
for i, el in enumerate(zip(obtain_result(2444485, nmf_reranker), obtain_result(2444485))):
    print(f"    Rank {i+1}     ")
#     print("------ NMF -----")
#     print(el[0].id)
#     print(el[0].name)
#     print(el[0].description)
#     print(el[0].revenue)
#     print(el[0].num_employees)
#     print(el[0].company_type)
#     print(el[0].sector)
    
    print("------ ES_more_like_this -----")
    
    for e in debug_es(el[1].id)["explanations"][0]["explanation"].split():
        print(e)

    print(el[1].id)
    print(el[1].name)
    print(el[1].description)
    print(el[1].revenue)
    print(el[1].num_employees)
    print(el[1].region)
    print("#"*80)

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    Rank 1     
------ ES_more_like_this -----
(company_description:copper
company_description:standard
company_description:boards
company_description:printed
company_description:aktiengesellschaft
company_description:rigid
company_description:and
company_description:austria
company_description:technologie
company_description:circuit
company_description:flexible
company_description:systemtechnik
company_description:substrates
company_description:hdi
company_description:pcbs
industry_class_code_desc.keyword:Printed
circuit
boards)~4
2444485
AT & S Austria Technologie & Systemtechnik Aktiengesellschaft
AT & S Austria Technologie & Systemtechnik Aktiengesellschaft, together with its subsidiaries, manufactures and distributes printed circuit boards (PCBs) worldwide. It operates through three segments: Mobile Devices & Substrates; Automotive, Industrial, Medical; and Others. The company offers double sided standard, metal core, and copper inlay PCBs; multilayer standard, high frequency/low 